# K Nearest Neighbors

In [1]:
import logging
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import(
    train_test_split,
    GridSearchCV
)
import sqlite3
import json
#Importar tuberia.
from sklearn.pipeline import Pipeline
#Importar pre procesador.
from sklearn.preprocessing import MinMaxScaler
#Importar algoritmo, regresion.
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
#Importar metricas (en caso de regresiones)
from sklearn.metrics import(
    mean_squared_error,
    mean_absolute_error,
    r2_score
)
from sklearn.inspection import permutation_importance
from sklearn.utils import resample

#Bitacora
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)

In [2]:
movies = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
credits = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [3]:
movies.sample(10,random_state=42)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
596,70000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,8427,"[{""id"": 810, ""name"": ""budapest""}, {""id"": 1930,...",en,I Spy,"When the Switchblade, the most sophisticated p...",13.267631,"[{""name"": ""Columbia Pictures Corporation"", ""id...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2002-10-31,33561137,97.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Attitude meets espionage.,I Spy,5.2,269
3372,7,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 28, ""n...",NaN,13006,"[{""id"": 3528, ""name"": ""flooding""}, {""id"": 9685...",en,Split Second,"In a futuristic London, the rising sea levels ...",4.857028,"[{""name"": ""Entertainment"", ""id"": 7676}, {""name...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",1992-05-01,5,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,He's seen the future. Now he has to kill it.,Split Second,5.7,63
2702,14000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 9648, ""na...",NaN,18041,"[{""id"": 3616, ""name"": ""college""}, {""id"": 9937,...",en,Gossip,"On a beautiful college campus, something ugly ...",5.833687,"[{""name"": ""Village Roadshow Pictures"", ""id"": 7...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2000-04-21,5108820,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"It can turn you on, or turn on you.",Gossip,5.5,68
2473,15000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",NaN,5038,"[{""id"": 107, ""name"": ""barcelona spain""}, {""id""...",en,Vicky Cristina Barcelona,Two girlfriends on a summer holiday in Spain b...,32.758254,"[{""name"": ""The Weinstein Company"", ""id"": 308},...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2008-08-15,96408652,96.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}, ...",Released,Life is the ultimate work of art,Vicky Cristina Barcelona,6.7,1020
8,250000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://harrypotter.warnerbros.com/harrypottera...,767,"[{""id"": 616, ""name"": ""witch""}, {""id"": 2343, ""n...",en,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...",98.885637,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2009-07-07,933959197,153.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Dark Secrets Revealed,Harry Potter and the Half-Blood Prince,7.4,5293
577,70000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ...",http://www.avp-movie.com/,395,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",en,AVP: Alien vs. Predator,When scientists discover something in the Arct...,42.957216,"[{""name"": ""Impact Pictures"", ""id"": 248}, {""nam...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",2004-08-12,171183863,101.0,"[{""iso_639_1"": ""it"", ""name"": ""Italiano""}, {""is...",Released,Whoever wins... We lose,AVP: Alien vs. Predator,5.5,1217
3172,9000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",NaN,6521,"[{""id"": 6078, ""name"": ""politics""}, {""id"": 9937...",en,The Contender,Political thriller about Laine Hanson's nomina...,7.029126,"[{""name"": ""Battleplan Productions"", ""id"": 2108...","[{""iso_3166_1"": ""DE"", ""name"": ""Germany""}, {""is...",2000-10-13,0,126.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Sometimes you can assassinate a leader without...,The Contender,6.7,60
811,55000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,1597,"[{""id"": 591, ""name"": ""cia""}, {""id"": 822, ""name...",en,Meet the Parents,"Greg Focker is ready to marry his girlfriend, ...",29.972240,"[{""name"": ""DreamWorks SKG"", ""id"": 

In [4]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [5]:
credits.sample(10, random_state=42)

,movie_id,title,cast,crew
596,8427,I Spy,"[{""cast_id"": 1, ""character"": ""Kelly Robinson"",...","[{""credit_id"": ""52fe44a8c3a36847f80a280d"", ""de..."
3372,13006,Split Second,"[{""cast_id"": 1, ""character"": ""Harley Stone"", ""...","[{""credit_id"": ""57011d5092514106c6004157"", ""de..."
2702,18041,Gossip,"[{""cast_id"": 1, ""character"": ""Beau Edson"", ""cr...","[{""credit_id"": ""58f82ecf9251415c5900238a"", ""de..."
2473,5038,Vicky Cristina Barcelona,"[{""cast_id"": 3, ""character"": ""Cristina"", ""cred...","[{""credit_id"": ""52fe43eec3a36847f80792ad"", ""de..."
8,767,Harry Potter and the Half-Blood Prince,"[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...","[{""credit_id"": ""52fe4273c3a36847f801fab1"", ""de..."
577,395,AVP: Alien vs. Predator,"[{""cast_id"": 7, ""character"": ""Alexa Woods"", ""c...","[{""credit_id"": ""52fe423ec3a36847f800f063"", ""de..."
3172,6521,The Contender,"[{""cast_id"": 1, ""character"": ""Sheldon Runyon"",...","[{""credit_id"": ""59991a7fc3a3686d67007a52"", ""de..."
811,1597,Meet the Parents,"[{""cast_id"": 1, ""character"": ""Gaylord \""Greg\""...","[{""credit_id"": ""557313ddc3a36838d3004484"", ""de..."
2077,19255,Away We Go,"[{""cast_id"": 1, ""character"": ""Burt Farlander"",...","[{""credit_id"": ""52fe47ca9251416c750a5843"", ""de..."
4032,20764,Sleep Dealer,"[{""cast_id"": 1, ""character"": ""Luz Mart\u00edne...","[{""credit_id"": ""5773e38f9251417fd40005e7"", ""de..."


In [6]:
conn = sqlite3.connect("../data/movies_database.db")

movies.to_sql("movies_table", conn, if_exists = "replace", index = False)
credits.to_sql("credits_table", conn, if_exists = "replace", index = False)

4803

In [7]:
query = """
    SELECT *
    FROM movies_table
    INNER JOIN credits_table
    ON movies_table.title = credits_table.title;
"""

df_raw = pd.read_sql_query(query, conn)
conn.close()

df_raw = df_raw.loc[:, ~df_raw.columns.duplicated()]
df_raw.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
def extract_json_field(json_str, field_name, default_value=None):
    try:
        data = json.loads(json_str)
        return [item[field_name] for item in data]
    except (TypeError, json.JSONDecodeError):
        return default_value

def extract_directors(json_str):
    try:
        data = json.loads(json_str)
        return " ".join([crew_member['name'] for crew_member in data if crew_member.get('job') == 'Director'])
    except (TypeError, json.JSONDecodeError):
        return None

# Procesar las columnas
df_baking = df_raw.copy()
df_baking["genres"] = df_baking["genres"].apply(lambda x: extract_json_field(x, "name", None))
df_baking["keywords"] = df_baking["keywords"].apply(lambda x: extract_json_field(x, "name", None))
df_baking["cast"] = df_baking["cast"].apply(lambda x: extract_json_field(x, "name", None)[:3] if x else None)
df_baking["crew"] = df_baking["crew"].apply(extract_directors)
df_baking["overview"] = df_baking["overview"].apply(lambda x: [x] if pd.notna(x) else None)

# Verificar los cambios
df_baking.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"[In the 22nd century, a paraplegic Marine is d...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,[A cryptic message from Bond’s past sends him ...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,[Following the death of District Attorney Harv...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"[John Carter is a war-weary, former military c...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [9]:
df_baking["overview"] = df_baking["overview"].apply(lambda x: [str(x)])
df_baking["genres"] = df_baking["genres"].apply(lambda x: [str(genre) for genre in x])
df_baking["keywords"] = df_baking["keywords"].apply(lambda x: [str(keyword) for keyword in x])
df_baking["cast"] = df_baking["cast"].apply(lambda x: [str(actor) for actor in x])
df_baking["crew"] = df_baking["crew"].apply(lambda x: [str(crew_member) for crew_member in x])

df_baking["tags"] = df_baking["overview"] + df_baking["genres"] + df_baking["keywords"] + df_baking["cast"] + df_baking["crew"]
df_baking["tags"] = df_baking["tags"].apply(lambda x: ",".join(x).replace(",", " "))

df_baking.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)

df_baking.iloc[0].tags


"['In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization.'] Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d Sam Worthington Zoe Saldana Sigourney Weaver J a m e s   C a m e r o n"

In [10]:
df = df_baking.copy()

In [11]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["tags"])

model = NearestNeighbors(n_neighbors = 7, algorithm = "brute", metric = "cosine")
model.fit(tfidf_matrix)

def get_movie_recommendations(movie_title):
    movie_index = df[df["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(df["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

input_movie = "Pirates of the Caribbean: At World's End"
recommendations = get_movie_recommendations(input_movie)
print("Film recommendations '{}'".format(input_movie))
for movie, distance in recommendations:
    print("- Film: {}".format(movie))

Film recommendations 'Pirates of the Caribbean: At World's End'
- Film: Pirates of the Caribbean: Dead Man's Chest
- Film: Pirates of the Caribbean: The Curse of the Black Pearl
- Film: Pirates of the Caribbean: On Stranger Tides
- Film: The Pirates! In an Adventure with Scientists!
- Film: Nim's Island
- Film: The Blue Lagoon
